# PA1

## Task1
We begin by using ....

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
  
# Read the CSV file.
data = pd.read_csv('CTG.csv', skiprows=1)

# Select the relevant numerical columns.
selected_cols = ['LB', 'AC', 'FM', 'UC', 'DL', 'DS', 'DP', 'ASTV', 'MSTV', 'ALTV',
                 'MLTV', 'Width', 'Min', 'Max', 'Nmax', 'Nzeros', 'Mode', 'Mean',
                 'Median', 'Variance', 'Tendency', 'NSP']
data = data[selected_cols].dropna()

# Shuffle the dataset.
data_shuffled = data.sample(frac=1.0, random_state=0)

# Split into input part X and output part Y.
X = data_shuffled.drop('NSP', axis=1)

# Map the diagnosis code to a human-readable label.
def to_label(y):
    return [None, 'normal', 'suspect', 'pathologic'][(int(y))]

Y = data_shuffled['NSP'].apply(to_label)

# Partition the data into training and test sets.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=0)

In [3]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier(strategy='most_frequent')

In [6]:
from sklearn.model_selection import cross_val_score
print('dummy accuracy:',cross_val_score(clf, Xtrain, Ytrain).mean())

dummy accuracy: 0.7805882352941176


Tree based classifiers:

In [7]:
from sklearn.tree import DecisionTreeClassifier as dtc
from sklearn.ensemble import RandomForestClassifier as rfc, GradientBoostingClassifier as gbc
dtc_clf = dtc() # without parameters ok?
rfc_clf = rfc() # without parameters ok?
gbc_clf = gbc() # without parameters ok?
print('Decision tree classifier accuracy:', cross_val_score(dtc_clf, Xtrain, Ytrain).mean())
print('Random forest classifier accuracy:', cross_val_score(rfc_clf, Xtrain, Ytrain).mean())
print('Gradient boosting classifier accuracy:', cross_val_score(gbc_clf, Xtrain, Ytrain).mean())

Decision tree classifier accuracy: 0.9217647058823528
Random forest classifier accuracy: 0.9400000000000001
Gradient boosting classifier accuracy: 0.9494117647058824


#### Linear classifiers

In [9]:
from sklearn.linear_model import Perceptron as perc, LogisticRegression as lr
from sklearn.svm import LinearSVC as lsvc
perc_clf = perc() 
#lr_clf = lr() # parameters needed.
#lsvc_clf = lsvc() # parameters needed.
print('perc', cross_val_score(perc_clf, Xtrain, Ytrain).mean())
#print('lr', cross_val_score(lr_clf, Xtrain, Ytrain).mean())
#print('lsvc', cross_val_score(lsvc_clf, Xtrain, Ytrain).mean())

perc 0.825294117647059
